In [5]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [6]:
# Grayson's path
#train_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\six-class\\data\\train\\"
#test_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\six-class\\data\\test\\"

# Will's path (comment out if not in use)
train_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\six-class\\data\\train\\"
test_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\six-class\\data\\test\\"

# Function to load and concatenate CSV files from a list of file paths
def load_and_concat(files, sample_fraction=None):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        if sample_fraction:
            df = df.sample(frac=sample_fraction, random_state=42)  # Random sampling
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# List all CSV files in the train and test folders
train_files = glob.glob(train_folder + "*.csv")
test_files = glob.glob(test_folder + "*.csv")

# Load and concatenate training and testing data
train_df = load_and_concat(train_files, sample_fraction=0.1)  # Use 10% of the data
test_df = load_and_concat(test_files, sample_fraction=0.1)

ValueError: No objects to concatenate

In [ ]:
# Combine target labels into a single column for multi-class classification
# Assuming the labels are one-hot encoded
target_columns = ['spoofing', 'benign', 'MQTT', 'recon', 'DDoS', 'DoS']
train_df['target'] = train_df[target_columns].idxmax(axis=1)
test_df['target'] = test_df[target_columns].idxmax(axis=1)

# Specify feature columns (excluding the target columns)
feature_columns = [col for col in train_df.columns if col not in target_columns + ['target']]

# Separate features and labels
X_train = train_df[feature_columns]
y_train = train_df['target']
X_test = test_df[feature_columns]
y_test = test_df['target']

In [ ]:
# Encode the target labels
y_train_encoded = pd.get_dummies(y_train)
y_test_encoded = pd.get_dummies(y_test)

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training features and transform both training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Build the DNN model
model = Sequential()
model.add(Dense(128, input_dim=len(feature_columns), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(target_columns), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the model
model.fit(X_train_scaled, y_train_encoded, epochs=25, batch_size=128, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test_encoded.values.argmax(axis=1)

# Evaluate the model
print(confusion_matrix(y_test_classes, y_pred_classes))
print(classification_report(y_test_classes, y_pred_classes, target_names=target_columns))

Epoch 1/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9012 - loss: 0.2970 - val_accuracy: 4.6084e-04 - val_loss: 13.8254
Epoch 2/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9275 - loss: 0.1850 - val_accuracy: 5.9350e-04 - val_loss: 14.3702
Epoch 3/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9279 - loss: 0.1822 - val_accuracy: 5.3066e-04 - val_loss: 15.7562
Epoch 4/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9293 - loss: 0.1785 - val_accuracy: 5.7954e-04 - val_loss: 16.8306
Epoch 5/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9284 - loss: 0.1796 - val_accuracy: 5.0273e-04 - val_loss: 18.0270
Epoch 6/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9292 - loss: 0.1776 - val_accuracy: 5.1670e-04 - val_loss: 16.0760
Epoch 7/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9295 - loss: 0.1765 - val_accuracy: 5.8652e-04 - val_loss: 17.8875
Epoch 8/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step